In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler , OneHotEncoder , PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error , explained_variance_score, max_error, median_absolute_error

In [2]:
data = pd.read_csv('../data/voiture_model.csv')
data

,etat_de_route,turbo,type_vehicule,roues_motrices,emplacement_moteur,marque,modele,empattement,longueur_voiture,largeur_voiture,poids_vehicule,taille_moteur,taux_alésage,chevaux,consommation_ville,consommation_autoroute,prix
0,3,standard,cabriolet,propulsion,avant,ALFA-ROMERO,GIULIA,225.0,428.8,162.8,1155.752416,130,88.1,111,11.20,8.71,13495.0
1,3,standard,cabriolet,propulsion,avant,ALFA-ROMERO,STELVIO,225.0,428.8,162.8,1155.752416,130,88.1,111,11.20,8.71,16500.0
2,1,standard,hayon,propulsion,avant,ALFA-ROMERO,QUADRIFOGLIO,240.0,434.8,166.4,1280.490216,152,68.1,154,12.38,9.05,16500.0
3,2,standard,berline,traction,avant,AUDI,100LS,253.5,448.6,168.1,1060.044504,109,81.0,102,9.80,7.84,13950.0
4,2,standard,berline,quatre_roues_motrices,avant,AUDI,100LS,252.5,448.6,168.7,1280.943808,136,81.0,115,13.07,10.69,17450.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,-1,standard,berline,propulsion,avant,VOLVO,145E(SW),277.1,479.6,175.0,1339.003584,141,96.0,114,10.23,8.40,16845.0
199,-1,turbo,berline,propulsion,avant,VOLVO,144EA,277.1,479.6,174.8,1383.002008,141,96.0,160,12.38,9.41,19045.0
200,-1,standard,berline,propulsion,avant,VOLVO,244DL,277.1,479.6,175.0,1366.219104,173,90.9,134,13.07,10.23,21485.0
201,-1,turbo,berline,propulsion,avant,VOLVO,246,277.1,479.6,175.0,1459.205464,145,76.5,106,9.05,8.71,22470.0


## Random Forest Regressor

In [3]:
# Séparer les variables indépendantes et la variable cible
X = data.drop('prix', axis=1)
y = data['prix']

# Créer un transformateur pour les variables numériques
num_transformer = Pipeline(steps=[
    ('scaler', MinMaxScaler()),
    ('poly', PolynomialFeatures(degree=2, include_bias=False))
])

# Créer un transformateur pour les variables catégorielles
cat_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Créer un préprocesseur pour combiner les deux transformateurs
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, ['empattement', 'longueur_voiture', 'largeur_voiture', 'poids_vehicule', 'taille_moteur', 'taux_alésage', 'chevaux', 'consommation_ville', 'consommation_autoroute']),
        ('cat', cat_transformer, ['etat_de_route', 'turbo', 'type_vehicule', 'roues_motrices', 'emplacement_moteur', 'marque', 'modele'])
    ])

# Créer le pipeline avec le préprocesseur et le modèle de régression par forêt aléatoire
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))
])

# Créer une grille de paramètres à tester
param_grid = {
    'regressor__n_estimators': [50, 100, 200],
    'regressor__max_depth': [None, 5, 10, 15],
    'regressor__min_samples_split': [2, 5, 10],
    'regressor__min_samples_leaf': [1, 2, 4],
    'regressor__max_features': ['auto', 'sqrt', 'log2'],
    'regressor__bootstrap': [True, False],
}

# Créer l'objet GridSearchCV
grid_search = GridSearchCV(model, param_grid, cv=5, n_jobs=-1)

# Diviser les données en ensembles de formation et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Effectuer la recherche des meilleurs paramètres
grid_search.fit(X_train, y_train)

# Prédire les prix pour l'ensemble de test avec le meilleur modèle trouvé
y_pred_forest = grid_search.predict(X_test)

# Évaluer les performances du modèle avec différentes métriques
print("Métriques d'évaluation du modèle :")
print("Meilleurs paramètres :", grid_search.best_params_)
print("Meilleur score :", grid_search.best_score_)
print("R2 Score :", r2_score(y_test, y_pred_forest))


Métriques d'évaluation du modèle :
Meilleurs paramètres : {'regressor__bootstrap': False, 'regressor__max_depth': None, 'regressor__max_features': 'sqrt', 'regressor__min_samples_leaf': 2, 'regressor__min_samples_split': 2, 'regressor__n_estimators': 100}
Meilleur score : 0.8929645543840763
R2 Score : 0.9146474584003123


## Linear Regression

In [4]:
from sklearn.linear_model import Ridge
from sklearn.compose import TransformedTargetRegressor

# Séparer les variables indépendantes et la variable cible
X = data.drop('prix', axis=1)
y = data['prix']

# Créer un transformateur pour les variables numériques
num_transformer = Pipeline(steps=[
    ('scaler', MinMaxScaler()),
    ('poly', PolynomialFeatures(degree=2, include_bias=False))
])

# Créer un transformateur pour les variables catégorielles
cat_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Créer un préprocesseur pour combiner les deux transformateurs
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, ['empattement', 'longueur_voiture', 'largeur_voiture', 'poids_vehicule', 'taille_moteur', 'taux_alésage', 'chevaux', 'consommation_ville', 'consommation_autoroute']),
        ('cat', cat_transformer, ['etat_de_route', 'turbo', 'type_vehicule', 'roues_motrices', 'emplacement_moteur', 'marque', 'modele'])
    ])

# Créer le pipeline avec le préprocesseur et le modèle de régression ridge
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', TransformedTargetRegressor(regressor=Ridge(alpha=1.0), transformer=MinMaxScaler()))
])

# Diviser les données en ensembles de formation et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entraîner le modèle sur les données d'entraînement
model.fit(X_train, y_train)

y_pred_ridge = model.predict(X_test)

# Évaluer les performances du modèle avec différentes métriques
print("Métriques d'évaluation du modèle :")
print("R2 Score :", r2_score(y_test, y_pred_ridge))

Métriques d'évaluation du modèle :
R2 Score : 0.8580501621459431


In [5]:
# Concaténer les prédictions des deux modèles dans une seule DataFrame
df_pred = pd.DataFrame({'Random Forest': y_pred_forest , 'Ridge Regression': y_pred_ridge})

# Ajouter la colonne "Actual" contenant les vraies valeurs de y_test
df = pd.concat([df_pred, pd.DataFrame({'Actual': y_test.values})], axis=1)
df


,Random Forest,Ridge Regression,Actual
0,30094.783750,26797.003590,30760.000
1,18762.493722,19816.699427,17859.167
2,13529.150333,14942.757252,16630.000
3,6775.468072,5296.311210,6669.000
4,10638.913333,13134.695322,18344.000
5,7238.957433,6759.091025,8916.500
6,10874.579167,11664.079264,11259.000
7,17862.085667,19407.617098,15998.000
8,19951.335833,21785.474220,21485.000
9,8214.278690,8355.729481,7775.000
